In [69]:
import argparse
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Lambda, Conv2D
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body
import numpy as np

In [62]:
def yolo_filter_boxes(box_confidence,boxes,box_class_probs,threshold = 0.6):
    
    box_scores = box_confidence*box_class_probs
    box_classes = K.argmax(box_scores,axis=-1)
    box_class_scores = K.max(box_scores,axis=-1,keepdims=False)
    
    filtering_mask = (box_class_scores>=threshold)
    scores = tf.boolean_mask(box_class_scores,filtering_mask)
    boxes = tf.boolean_mask(boxes,filtering_mask)
    classes = tf.boolean_mask(box_classes,filtering_mask)
    
    return scores,boxes,classes

In [65]:
with tf.Session() as test:
    
    box_confidence = tf.random_normal([19,19,5,1],mean=1,stddev=4,seed=1)
    boxes = tf.random_normal([19,19,5,4],mean=1,stddev=4,seed=1)
    box_class_probs = tf.random_normal([19,19,5,80],mean=1,stddev=4,seed=1)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.5)
    print("scores[4] = " + str(scores[4].eval()))
    print("boxes[4] = " + str(boxes[4].eval()))
    print("classes[4] = "+str(classes[4].eval()))
    


scores[4] = 12.057256
boxes[4] = [-5.9719086 -0.7515718 -1.26404    2.2826772]
classes[4] = 5


In [72]:
# Non-Max Suppression
# Intersection over Union


def iou(box1,box2):
    
    (box1_x1,box1_y1,box1_x2,box1_y2) = box1
    (box2_x1,box2_y1,box2_x2,box2_y2) = box2
    
    xi1 = np.maximum(box1[0],box2[0])
    yi1 = np.maximum(box1[1],box2[1])
    xi2 = np.minimum(box1[2],box2[2])
    yi2 = np.minimum(box1[3],box2[3])
    inter_width = xi2-xi1
    inter_height = yi2-yi1
    
#     case in which they doesn't intersect, then inter_height or inter_width will be 0 and intersection area will return 0

    intersection_area = max(inter_width,0)*max(inter_height,0)
    
#     Union_Area
    
    box1_area = (box1[2]-box1[0])*(box1[3]-box1[1])
    box2_area = (box2[2]-box2[0])*(box2[3]-box2[1])
    union_area = box1_area+box2_area-intersection_area
    
    iou = float(intersection_area)/float(union_area)
    
    
    return iou
    
    

In [73]:
# Test case run
box1 = (2,1,4,3)
box2 = (1,2,3,4)
print(iou(box1,box2))

0.14285714285714285
